In [1]:
import sys
paths = ['/Users/jacob/Desktop/docs/kaggle/two_sigma/_b_Market/scripts',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/quickpipeline',
        '/Users/jacob/Desktop/docs/ML/_a_python/_1_code/notebooks/pd_feature_union']

for path in paths:
    sys.path.append(path)

from market_imports import *
from market_code import *
from market_trans import *
from quickpipe_mod import * 
from pandas_feature_union import *

data_path = '/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/data'
df_market = pd.read_csv(data_path + '/marketdata_sample.csv') 
df_news = pd.read_csv(data_path + '/news_sample.csv')


import warnings
warnings.filterwarnings(action='ignore')

import datetime as dt

In [26]:
# csv = pd.read_csv(data_path + '/sigma_data.csv', index_col='Date', parse_dates=True, 
#                  infer_datetime_format=True)

csv = pd.read_csv(data_path + '/big.csv', index_col='Date', parse_dates=True, 
                 infer_datetime_format=True)

# COMPUTE RESIDUAL FOR RESPONSE
import statsmodels.tsa.api as tsa
from statsmodels.tsa.seasonal import seasonal_decompose 

x = csv[['returns_open_raw10_next']]
# Freq will depend on asset number - this needs to be researched
result = seasonal_decompose(x, model='additive', freq=50)

csv['returnsOpenNextMktres10'] = result.resid
# Lose the first and last day
csv = csv[-csv['returnsOpenNextMktres10'].isnull()]
csv.drop(['returns_open_raw10_next'], axis=1, inplace=True)

check = csv.columns[:4].tolist()
csv.drop_duplicates(check, inplace=True)

# ELC is insanely high. The next highest price is around 300
csv[csv.Close ==csv.Close.max()]
csv[csv.Close > 1000]
csv = csv[csv.asset != 'ELC']

# Matching the kernel data some more
csv.index.name = 'time'
csv.reset_index(inplace=True)
csv.rename(columns={'asset': 'assetCode', 'Volume': 'volume',
                  'Close': 'close'}, inplace=True)

print(csv.shape)
csv.head()

(2121121, 10)


,time,assetCode,Open,close,volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,returnsOpenNextMktres10
0,2013-01-16,EQC,16.00,16.049999,537700.0,0.004371,0.008158,-0.000623,-0.000625,-0.020582
1,2013-01-16,BDN,12.50,12.820000,2712500.0,0.004691,-0.003992,0.051211,0.020203,0.004975
2,2013-01-16,FBHS,31.00,30.870001,730700.0,-0.004202,-0.000967,0.004546,0.035794,-0.008944
3,2013-01-16,NMFC,14.65,14.740000,105100.0,0.003398,-0.009511,-0.016151,-0.023610,0.014740
4,2013-01-16,SMLP,19.67,19.660000,59000.0,0.001527,0.003565,-0.026106,-0.018137,-0.079554


__Notes:__  Assuming that we have grouped by assets - multi-index

In [146]:
qcom = csv.query("assetCode == 'QCOM'")['close']
qcom.shape

(988,)

## Test 1

In [33]:
# Proposed method
X = qcom.values
x = np.arange(qcom.shape[0])
# this is an index array
def sma(i, n=20):
    return np.nansum(X[i-20+1:i+1]) / n

vf = np.vectorize(sma)
print(vf(x)[-5:])

%time

[67.14549866 67.22499886 67.39549866 67.4349987  67.52149849]
CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


In [37]:
# Pandas method

sma = qcom.rolling(20, min_periods=20).mean()
 # match up but seems to be an extra value, corrected with the i+1
%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [38]:
print(sma.tail())

2110829    67.145499
2113510    67.224999
2116513    67.395499
2118676    67.434999
2119720    67.521498
Name: close, dtype: float64


## Test 2 -> we have a winner. Lambdas to the graveyard

In [36]:
# Proposed method
X = qcom.values
x = np.arange(qcom.shape[0])
# this is an index array


def bollingerBands(i, n=20):
    sma = np.nansum(X[i-n+1:i+1]) / n
    std = np.std(X[i-n+1:i+1])
    return sma, sma + 2 * std, sma - 2 * std

vf = np.vectorize(bollingerBands)
sma, U, L = vf(x)

%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [39]:
# Proposed method
X = qcom.values
x = np.arange(qcom.shape[0])
# this is an index array


def bollingerSqueeze(i, n=20):
    sma = np.nansum(X[i-n+1:i+1]) / n
    std = np.std(X[i-n+1:i+1])
    U = sma + (std*2)    
    L = sma - (std*2) 
    squeeze = (U - L)/ sma
    
    return squeeze, U, L

vf = np.vectorize(bollingerBands)
sq, U, L = vf(x)

%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [45]:

sd_20 = (qcom.
        rolling(window=20, min_periods=20).
        std().
        values  
        )


sma_20 = (
        qcom.
        rolling(window=20, min_periods=20).
        mean().
        values
        )


# convert everything to arrays   

U = sma_20 + (sd_20*2)    
L = sma_20 - (sd_20*2) 
squeeze = (U - L)/ sma_20

%time

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs


## Multi-index vs grouping

In [190]:
X = csv.copy()

In [180]:
def bBands(x, n=20):
    """This works. The transform method is taking this function and 
    interating it over every row within each group. Which is bizzare. Since
    this index should only work for windows"""
    return np.nansum(x[-n:]) / n
    

In [185]:
def bbSqueeze(X):
    
    '''price moves toward upper indicate bullish. This
    returns the distance from stock price to upper or lower 
    BB. If the distance turns negative, the price is above/below. 
    See indicator lambdas.
    
    Squeeze is calculate per asset. However, comparing it to the 6 month min is a bit
    more involved. For now, leave it external. It can be integrated into the func/class
    later if it is useful. Notice the shift on the rolling min(). The present squeeze value
    should not be included in the window we are comparing it with.
    '''
    
    asset_close = X.groupby('assetCode')['close']
    
    sma = asset_close.transform(lambda x: np.nansum(x[-20:]) / 20)
    std = asset_close.transform(lambda x: np.std(x[-20:]) / 20)
       
    U = sma + (std*2)    
    L = sma - (std*2) 
    
    del asset_close
    
    return (U - L)/ sma, U, L
    

In [186]:
df_ = bbSqueeze(X)

In [188]:
sq, u, l = df_

In [189]:
sq[-5:]

2121116    0.007921
2121117    0.009022
2121118    0.003359
2121119    0.009012
2121120    0.006576
Name: close, dtype: float64

In [192]:
squeeze[-5:]

array([0.16254558, 0.18512325, 0.06891636, 0.18493176, 0.13492956])

In [182]:
asset_close = X.groupby('assetCode')['close']
    
sma = asset_close.transform(lambda x: np.nansum(x[-20:]) / 20)

In [191]:
def BB_distance(dframe):
    
    '''price moves toward upper indicate bullish. This
    returns the distance from stock price to upper or lower 
    BB. If the distance turns negative, the price is above/below. 
    See indicator lambdas.
    
     Squeeze is calculate per asset. However, comparing it to the 6 month min is a bit
    more involved. For now, leave it external. It can be integrated into the func/class
    later if it is useful. Notice the shift on the rolling min(). The present squeeze value
    should not be included in the window we are comparing it with.
    '''
    
    asset_close = dframe.groupby('assetCode')['close']
    
    sd_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            std()).
            values        
            )
    
    sma_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            mean()).
            values
            )
    
    # convert everything to arrays
    
    close = dframe['close'].values   
    U = sma_20 + (sd_20*2)    
    L = sma_20 - (sd_20*2) 
    squeeze = (U - L)/ sma_20

    
#     return (U - close, close - L, squeeze)
    return (U, L, squeeze)

U, L,  squeeze = BB_distance(X)
    
    
# data['BB_upper_dis'], data['BB_lower_dis'],  data['squeeze']= BB_distance(data)
    
    


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [170]:
X = csv.copy()
# X.reset_index(inplace=True)
groups = X.groupby('assetCode')['close']

sma = groups.transform(bBands)

%time

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [173]:
sma_20[-5:]

array([12.80499997, 19.26400003, 98.04650116, 60.30749989, 40.13950005])

In [171]:
sma[-5:]

2121116    12.805000
2121117    19.264000
2121118    98.046501
2121119    60.307500
2121120    40.139500
Name: close, dtype: float64

In [ ]:
# Proposed method






x = np.arange(df.shape[0])
# this is an index array
def sma(i, n=20):
    
    X = mult['close'].loc[assets[i]].values
    return np.nansum(X[i-n+1:i+1]) / n

vf = np.vectorize(sma)
# print(vf(x)[-5:])

sma = vf(x)

%time

In [90]:
x.shape

(2121121,)

In [ ]:
X = csv.copy()

asset_close = X.groupby('assetCode')['close']
    
    
    sma_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            mean()).
            values
            )